In [ ]:
#from google.colab import drive
#drive.mount("/content/drive")

In [ ]:
#cd "/content/drive/MyDrive/AML_Assignment_2_2021"

In [ ]:
ls

In [ ]:
"""
Created on Thu Nov 11 17:09:22 2021

@author: marco
"""

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np

import matplotlib.pyplot as plt

from ex3_convnet_utils import get_dataset_loaders, weights_init, PrintModelSize, VisualizeFilter
from ex3_convnet_utils import ConvNet
from ex3_convnet_utils import complete_training_and_validation
from ex3_convnet_utils import test_model

import pandas as pd

gridsearch_data_default_path = "ex3_data_augmentation_complete.csv"

gridsearch_data = pd.DataFrame()

try:
    gridsearch_data = pd.read_csv(gridsearch_data_default_path, sep = ';', index_col = None)
    print("Number of lines in csv: ", len(gridsearch_data))
    
except Exception as E:
    print(E)
    print("No worries! Seems like this is the first run, or maybe the file is actually not there.")


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

index_last_checkpoint = len(gridsearch_data.index) # Will use this to get to the last permutation of hyperparameters
    
    
cols = ['transformations',
        'best_model_valid_accuracy',
        'early_stopped_valid_accuracy',
        'best_model_test_accuracy',
        'early_stopped_best_accuracy']


data_aug_transforms = []

input_size = 3
num_classes = 10
hidden_size = [128, 512, 512, 512, 512]
num_training = 49000
num_validation = 1000
norm_layer = 'BN'

num_epochs = 30
batch_size = 200
learning_rate = 0.001
learning_rate_decay = 0.9
reg = 0.005


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

current_iteration = 0



all_transformations = [

                        [transforms.RandomHorizontalFlip(p=0.3)],
                        [transforms.RandomHorizontalFlip(p=0.6)],

                        [transforms.RandomRotation(20)],
                        [transforms.RandomRotation(60)],
                        [transforms.RandomRotation(110)],


                        [transforms.RandomHorizontalFlip(p=0.3), transforms.RandomRotation(20)],
                        [transforms.RandomHorizontalFlip(p=0.6), transforms.RandomRotation(60)],
                        [transforms.RandomHorizontalFlip(p=0.3), transforms.RandomRotation(110)],
                        [transforms.RandomHorizontalFlip(p=0.6), transforms.RandomRotation(20)],
                        [transforms.RandomHorizontalFlip(p=0.3), transforms.RandomRotation(60)],
                        [transforms.RandomHorizontalFlip(p=0.6), transforms.RandomRotation(110)],

                        [transforms.RandomPerspective(distortion_scale=0.2, p=1.0)],
                        [transforms.RandomPerspective(distortion_scale=0.5, p=1.0)],
                        [transforms.RandomPerspective(distortion_scale=0.7, p=1.0)],

                        [transforms.RandomPerspective(distortion_scale=0.4, p=1.0),
                        transforms.RandomRotation(20)],



                        [transforms.RandomPerspective(distortion_scale=0.7, p=1.0),
                        transforms.RandomRotation(20)],



                        [transforms.RandomPerspective(distortion_scale=0.4, p=1.0),
                        transforms.RandomRotation(60)],



                        [transforms.RandomPerspective(distortion_scale=0.7, p=1.0),
                        transforms.RandomRotation(60)],

                        [transforms.RandomPerspective(distortion_scale=0.4, p=1.0),
                        transforms.RandomRotation(110)]
    
                      ]


for data_aug_transforms in all_transformations: 

  if current_iteration >= index_last_checkpoint: # Using the rows of the dataframe, I can go back to the last combination of parameters
      


      ''' Get the transformed datasets '''
      train_loader, val_loader, test_loader = get_dataset_loaders(data_aug_transforms,
                                              batch_size,
                                              num_training, 
                                              num_validation)

      

      ''' Train on the transformed dataset '''
      model = ConvNet(input_size, hidden_size, num_classes, norm_layer=norm_layer).to(device)
      model.apply(weights_init)     
      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg)
      
      

      ''' Get the results '''
      results = complete_training_and_validation(model = model,
                                      num_epochs = num_epochs,
                                      train_loader = train_loader,
                                      val_loader = val_loader,
                                      device = device,
                                      learning_rate = learning_rate,
                                      learning_rate_decay = learning_rate_decay,
                                      reg = reg,
                                      batch_size = batch_size)
      

      ''' Save to csv file '''
      
      best_model = results[0]
      early_stopped_model = results[1]
      best_model_accuracy = results[4]
      early_stopped_accuracy = results[5]
      
      best_model_test_accuracy = np.nan
      early_stopped_test_accuracy = np.nan
      
      if best_model != None:
          best_model_test_accuracy = test_model(best_model, test_loader, device)
          print("Best model test accuracy: ", best_model_test_accuracy)
          
      if early_stopped_model != None:
          early_stopped_test_accuracy = test_model(early_stopped_model, test_loader, device)
          print("Early stopped test accuracy: ", early_stopped_test_accuracy)
      

      new_row = pd.DataFrame(data = [[str(data_aug_transforms), best_model_accuracy, early_stopped_accuracy, best_model_test_accuracy, early_stopped_test_accuracy]], columns = cols)
      
      gridsearch_data = gridsearch_data.append(new_row, ignore_index = True)
      
      gridsearch_data.to_csv(gridsearch_data_default_path, sep = ';', na_rep = 'nan', index = False)                     
      print("\n")
  else:
      print("Skipping iteration, currently at: ", current_iteration)
      
  current_iteration += 1